In [1]:
"""Celery tasks for photos"""
from celery.decorators import periodic_task
from celery.utils.log import get_task_logger
from datetime import timedelta
from django.conf import settings
import os
import glob

IMAGE_STAGING = os.path.join(settings.STAGING_ROOT, 'STAGING', 'IMAGES')
PDF_STAGING = os.path.join(settings.STAGING_ROOT, 'STAGING', 'IMAGES')
def get_staging_pdf_files(self, magazine='1'):
    globpattern = '{folder}/UNI1{version}VER*.pdf'.format(
        folder=PDF_STAGING,
        version=magazine,
    )
    all_files = glob(globpattern)
    new_files = []
    for pdf_file in all_files:
        age = datetime.now() - \
            datetime.fromtimestamp(os.path.getctime(pdf_file))
        if age.days > 4:
            os.remove(pdf_file)
        else:
            new_files.append(pdf_file)
    return sorted(new_files)

In [2]:
import os
from datetime import datetime

now = datetime.now()
IMAGE_STAGING = os.path.join(settings.STAGING_ROOT, 'STAGING', 'IMAGES')
os.chdir(IMAGE_STAGING)
print(os.path.abspath(os.curdir))

/srv/fotoarkiv_universitas/STAGING/IMAGES


In [18]:
for file_path in sorted(os.listdir(), key=lambda f: os.path.getmtime(f), reverse=True): 
#    exists = ImageFile.objects.filter(old_file_path=file_path)
#    if exists:
#        print(exists)
#        continue
    mtime = os.path.getmtime(file_path)
    modified = datetime.fromtimestamp(mtime)
    print('{0:<40.40}  {1}'.format(file_path, modified))
    

Nummer-14-onsdag-18-oktober1989.jpg       2015-10-14 11:32:00
28-SID-oyeblikk-03-AOB.jpg                2015-10-14 11:31:58
statsbudsjett.jpg                         2015-10-14 11:31:58
28-SID-oyeblikk-02-AOB.jpg                2015-10-14 11:31:57
28-SID-oyeblikk-01-AOB.jpg                2015-10-14 11:31:56
28-NYH-nyhrektorkje-03-XX.jpg             2015-10-14 11:31:54
28-NYH-nyhrektorkje-02-XX.jpg             2015-10-14 11:31:54
Arkivfoto-uio-rektor-KjetilBlom.jpg       2015-10-14 11:31:54
28-NYH-nyhrektorkje-01-XX.jpg             2015-10-14 11:31:52
26-NYH-nyharbeidsmo-03-HB.jpg             2015-10-14 11:31:52
26-NYH-nyharbeidsmo-02-HB.jpg             2015-10-14 11:31:51
26-NYH-nyharbeidsmo-01-HB.jpg             2015-10-14 11:31:50
26-NYH-nyharbeidsmo-07-HB.jpg             2015-10-14 11:31:49
28-NYH-karakterkaos-03-KEK-Copy.jpg       2015-10-14 11:31:47
06-NYH-nyhnsovalg-03-AN.jpg               2015-10-14 11:31:46
06-NYH-nyhnsovalg-04-AN.jpg               2015-10-14 11:31:45
26-NYH-n